In [1]:
# sdkvm

In [1]:
import pandas as pd
import joblib
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report

In [6]:
train_df = pd.read_csv(".././Training-Testing Dataset/Training Data (RWS Oversampling).csv")
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")
target_col = "target"

X_train = train_df.drop(columns=[target_col])
X_test = test_df.drop(columns=[target_col])

y_train = train_df[target_col]
y_test  = test_df[target_col]

print(y_train.value_counts())
categorical_columns = [col for col in X_train.columns if X_train[col].dtype == 'object']

for col in categorical_columns:
    X_train[col] = X_train[col].astype("category")
    X_test[col] = X_test[col].astype("category")

#LGBM only accepts numerical data in the target columns
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

train_set = lgbm.Dataset(X_train,label=y_train,categorical_feature=categorical_columns)
test_set = lgbm.Dataset(X_test,label=y_test,categorical_feature=categorical_columns)


target
Medium_Yes    4492
Low_No        4492
Medium_No     4492
High_No       4492
Low_Yes       4492
High_Yes      4492
Name: count, dtype: int64


In [9]:
print(len(set(y_train)))

6


In [7]:
params = {
    'objective':'multiclass',
    'num_class':len(set(y_train)),
    'metric':'multi_error',
    'learning_rate':0.09,
    'num_leaves':170,
    'max_depth':-1,
    'verbose':-1
}

model = lgbm.train(params,train_set,valid_sets=[test_set],num_boost_round=500)

In [8]:
# Predict
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.7179588607594937
              precision    recall  f1-score   support

           0       0.58      0.48      0.52       509
           1       0.89      0.98      0.93       304
           2       0.67      0.55      0.61       509
           3       0.87      1.00      0.93       188
           4       0.52      0.60      0.56       509
           5       0.92      0.98      0.95       509

    accuracy                           0.72      2528
   macro avg       0.74      0.77      0.75      2528
weighted avg       0.71      0.72      0.71      2528



In [12]:
joblib.dump(model,"lightGBM_model.pkl")

['lightGBM_model.pkl']

In [3]:
model = joblib.load("lightGBM_model.pkl")

In [13]:
test_df = pd.read_csv(".././Training-Testing Dataset/Testing data Undersampled(All).csv")

target_col = "target"   

X_test = test_df.drop(columns=[target_col])

y_test  = test_df[target_col]

print(y_test.value_counts())

for col in categorical_columns:
    X_test[col] = X_test[col].astype("category")

y_test = le.fit_transform(y_test)


target
Medium_Yes    4492
Low_No        4492
Medium_No     4492
High_No       4492
Low_Yes       4492
High_Yes      4492
Name: count, dtype: int64


In [14]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.9188557435440784
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      4492
           1       0.93      0.98      0.95      4492
           2       0.90      0.90      0.90      4492
           3       0.93      1.00      0.97      4492
           4       0.94      0.80      0.86      4492
           5       0.92      0.98      0.95      4492

    accuracy                           0.92     26952
   macro avg       0.92      0.92      0.92     26952
weighted avg       0.92      0.92      0.92     26952



In [24]:
sum_importance = sum(model.feature_importance())
for i,j in zip(model.feature_name(), model.feature_importance()):
    print(f"{i}:{(j/sum_importance)*1000 : 0.2f}")

Age_at_Marriage: 404.68
Gender: 92.20
Education_Level: 17.28
Caste_Match: 65.95
Religion: 14.06
Urban_Rural: 83.51
Income_Level: 127.19
Spouse_Working: 89.78
Inter-Caste: 60.75
Inter-Religion: 44.60
